In [1]:
import pandas as pd
import numpy as np
import pickle

In [4]:
sequences = {
    "A":
    [
        "HNPVVMVHGMGGASYNFASIKSYLV",
        "HNPVVMVHGMGGASYNFASIKSYLV",
        "HNPVVMVHGMGGASYNFASIKSYLV",
        "HNPVVMVHGMGGASYNFASIKSYLV",
        "HNPVVMVHGMGGASYNFASIKSYLV"
    ]
}
with open("sequences.pkl", "wb") as f:
    pickle.dump(sequences, f)

In [3]:
seq_index = [0,1,2,3,4]
sequence = ["HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV", "HNPVVMVHGMGGASYNFASIKSYLV"]
iteration = [1,2,3,4,5]
metric1 = [-2, -6, -1, -1, -5]
metric2 = [-1, -6, -1, -3, -5]
rank = [1, 2, 3, 4, 5]
df = pd.DataFrame({'seq_index': seq_index, 'Sequence': sequence, 'iteration': iteration, 'metric1': metric1, 'metric2': metric2, 'Rank': rank})
dataframes = {"A": df}
with open("data_frame.pkl", "wb") as f:
    pickle.dump(dataframes, f)